Nhấn Ctrl+F9 để chạy tất cả

In [1]:
!pip -qq install -U xformers --index-url https://download.pytorch.org/whl/cu118
!pip -qq install langchain optimum qdrant-client wikipedia
!pip -qq install https://github.com/vllm-project/vllm/releases/download/v0.2.2/vllm-0.2.2+cu118-cp310-cp310-manylinux1_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.2/182.2 kB 25.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.5 MB

In [2]:
GENERATE_MODEL_NAME="phatjk/vietcuna-7b-v3-AWQ"
EMBEDDINGS_MODEL_NAME="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
QDRANT_URL = "<QDRANT_URL>"
QDRANT_COLLECTION_NAME = "nttu_sotay_vector_db_v1"

In [3]:
#@title API KEY OF DATABASE
HUGGINGFACE_API_KEY = "<HUGGINGFACE_API_KEY>"
QDRANT_API_KEY =      "<QDRANT_API_KEY>"

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from optimum.bettertransformer import BetterTransformer
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_rerank = AutoModelForSequenceClassification.from_pretrained('amberoad/bert-multilingual-passage-reranking-msmarco').to(device)
model_rerank = BetterTransformer.transform(model_rerank)
tokenizer_rerank = AutoTokenizer.from_pretrained('amberoad/bert-multilingual-passage-reranking-msmarco')

config.json:   0%|          | 0.00/696 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [5]:
from langchain.schema.document import Document
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.retrievers import WikipediaRetriever
from typing import List
class RerankRetriever(VectorStoreRetriever):
    vectorstore: VectorStoreRetriever
    def get_relevant_documents(self, query: str) -> List[Document]:
        docs = self.vectorstore.get_relevant_documents(query=query)
        candidates = [doc.page_content for doc in docs]
        queries = [query]*len(candidates)
        features = tokenizer_rerank(queries, candidates,  padding=True, truncation=True, return_tensors="pt").to(device)
        with torch.no_grad():
            scores = model_rerank(**features).logits
            values, indices = torch.sum(scores, dim=1).sort()
            # relevant_docs = docs[indices[0]]
        return [docs[indices[0]],docs[indices[1]]]
class RerankWikiRetriever(VectorStoreRetriever):
    vectorstore: WikipediaRetriever
    def get_relevant_documents(self, query: str) -> List[Document]:
        docs = self.vectorstore.get_relevant_documents(query=query)
        candidates = [doc.page_content for doc in docs]
        queries = [query]*len(candidates)
        features = tokenizer_rerank(queries, candidates,  padding=True, truncation=True, return_tensors="pt").to(device)
        with torch.no_grad():
            scores = model_rerank(**features).logits
            values, indices = torch.sum(scores, dim=1).sort()
            # relevant_docs = docs[indices[0]]
        return [docs[indices[0]],docs[indices[1]]]

In [6]:
from langchain.retrievers import WikipediaRetriever
from langchain.vectorstores import Qdrant
from langchain.llms import HuggingFacePipeline
from qdrant_client import QdrantClient
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.chains import RetrievalQA,MultiRetrievalQAChain
from langchain.llms import VLLM
from langchain.llms import HuggingFaceHub

class LLMServe:
    def __init__(self) -> None:
      self.embeddings = self.load_embeddings()
      self.current_source = "wiki"
      self.retriever = self.load_retriever(retriever_name = self.current_source,embeddings=self.embeddings)
      self.pipe = self.load_model_pipeline(max_new_tokens=300)
      self.prompt = self.load_prompt_template()
      self.rag_pipeline = self.load_rag_pipeline(llm=self.pipe,
                                            retriever=self.retriever,
                                            prompt=self.prompt)
    def load_embeddings(self):
      embeddings = HuggingFaceInferenceAPIEmbeddings(
          model_name=EMBEDDINGS_MODEL_NAME,
          api_key = HUGGINGFACE_API_KEY,
          model_kwargs = {'device': "auto"}
      )
      return embeddings

    def load_retriever(self,retriever_name,embeddings):
      retriever=None
      if retriever_name == "wiki":
        retriever = RerankWikiRetriever(vectorstore = WikipediaRetriever(lang="vi",
                                       doc_content_chars_max=800,top_k_results=15))
      else:
        client = QdrantClient(
            url=QDRANT_URL,api_key=QDRANT_API_KEY, prefer_grpc=False
        )
        db = Qdrant(client=client,
                    embeddings=embeddings,
                    collection_name=QDRANT_COLLECTION_NAME)

        retriever = RerankRetriever(vectorstore = db.as_retriever(search_kwargs={"k":15}))

      return retriever

    def load_model_pipeline(self,max_new_tokens=100):
      llm = VLLM(
          model=GENERATE_MODEL_NAME,
          trust_remote_code=True,  # mandatory for hf models
          max_new_tokens=max_new_tokens,
            # temperature=1.0,
            # top_k=50,
            # top_p=0.9,
          top_k=10,
          top_p=0.95,
          temperature=0.4,
          dtype="half",
          vllm_kwargs={"quantization": "awq"}
      )
      return llm

    def load_prompt_template(self):
      # query_template = "Bạn là một trợ lý của trường Đại học Nguyễn Tất Thành. Hãy trả lời câu hỏi sau dựa trên ngữ cảnh, nếu ngữ cảnh không cung cấp câu trả lời hoặc không chắc chắn hãy trả lời 'Tôi không biết thông tin này, tuy nhiên đoạn thông tin dưới phần tham khảo có thể có câu trả lời cho bạn!' đừng cố tạo ra câu trả lời không có trong ngữ cảnh.\nNgữ cảnh: {context} \nCâu hỏi: {question}\nTrả lời: "
      # query_template = "Tham khảo ngữ cảnh:{context}\n\n### Câu hỏi:{question}\n\n### Trả lời:"
      query_template = "Bạn là một chatbot thông minh trả lời câu hỏi dựa trên ngữ cảnh (context).\n\n### Context:{context} \n\n### Human: {question}\n\n### Assistant:"
      prompt = PromptTemplate(template=query_template,
                        input_variables= ["context","question"])
      return prompt

    def load_rag_pipeline(self,llm,retriever,prompt):
      rag_pipeline = RetrievalQA.from_chain_type(
      llm=llm, chain_type='stuff',
      retriever=retriever,
      chain_type_kwargs={
      "prompt": prompt
      },
      return_source_documents=True)
      return rag_pipeline

    def rag(self,source):
      if source == self.current_source:
        return self.rag_pipeline
      else:
        self.retriever = self.load_retriever(retriever_name=source,embeddings=self.embeddings)
        self.rag_pipeline = self.load_rag_pipeline(llm=self.pipe,
                                      retriever=self.retriever,
                                      prompt=self.prompt)
        self.current_source = source
        return self.rag_pipeline

In [7]:
app = LLMServe()

config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

WARNING 12-05 12:49:17 config.py:140] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 12-05 12:49:17 llm_engine.py:72] Initializing an LLM engine with config: model='phatjk/vietcuna-7b-v3-AWQ', tokenizer='phatjk/vietcuna-7b-v3-AWQ', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=awq, seed=0)


tokenizer_config.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.25G [00:00<?, ?B/s]

INFO 12-05 12:51:06 llm_engine.py:207] # GPU blocks: 588, # CPU blocks: 546


In [30]:
# 1
print(app.rag(source="nttu")("Điều kiện nhận học bổng?")["result"])

Processed prompts: 100%|██████████| 1/1 [00:09<00:00,  9.66s/it]

 Điều kiện nhận học bổng khuyến khích học tập bao gồm:

1. Sinh viên hoàn thành tất cả các học phần đã đăng ký trong năm học (gồm cả 03 học kỳ trong năm học);
2. Điểm trung bình chung của năm học đạt từ 2.50 trở lên theo thang điểm 4;
3. Sinh viên đạt tổng số tín chỉ tích lũy từ 30 tín chỉ trở lên trong năm học, riêng với SV năm cuối đạt tổng số tín chỉ tích lũy từ 20 tín chỉ trở lên trong năm học;
4. Sinh viên không có điểm thi kết thúc môn học và điểm trung bình tổng kết của mỗi học phần dưới 5.0 điểm trong năm học;
5. Kết quả rèn luyện năm học từ loại khá trở lên (>=70 điểm);
6. Sinh viên không được xét kết quả các học phần miễn học, chuyển điểm vào điểm trung bình chung của năm học;
7. Sinh viên tạm nghỉ học trong năm học xét học bổng sẽ không được xét;

Để đạt học bổng, sinh viên cần có học lực và điểm rèn luyện từ loại khá trở lên.


In [16]:
# 2
print(app.rag(source="nttu")("Bao nhiêu điểm thì học lực Xuất sắc?")["result"])

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.07s/it]

 Học lực Xuất sắc có điểm từ 3,60 đến 4,00.


In [17]:
# 3
print(app.rag(source="nttu")("Bao nhiêu điểm thì học lực Giỏi?")["result"])

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.72s/it]

 Học lực Giỏi được xác định dựa trên điểm hệ 4 từ 3.20 đến 3.59.


In [32]:
# 4
print(app.rag(source="nttu")("Bao nhiêu điểm thì học lực Khá?")["result"])

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.12s/it]

 Học lực Khá có điểm từ 2,50 đến 3,19.


In [19]:
# 5
print(app.rag(source="nttu")("Điều kiện thực tập tốt nghiệp là gì?")["result"])

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.29s/it]

 Điều kiện thực tập tốt nghiệp là sinh viên phải hoàn thành tối thiểu 60% số tín chỉ trong chương trình đào tạo. Tùy theo khoa tỷ lệ phần trăm số tín chỉ sinh viên cần phải hoàn thành có thể cao hơn. Sinh viên không được nợ các học phần chuyên ngành. Điều kiện thực tập tốt nghiệp được các khoa thông báo cụ thể đến sinh viên ngay từ đầu khóa học hoặc chậm nhất tại thời điểm khoa chia chuyên ngành đào tạo.


In [20]:
# 6
print(app.rag(source="nttu")("Học phần đã đăng ký có trạng thái N* là gì?")["result"])

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.24s/it]

 Trạng thái N* khi đăng ký học phần là trạng thái mà sinh viên đã đăng ký học phần nhưng không đóng học phí học phần đã đăng ký sau khi đã hết hạn lần thứ hai. Sinh viên phải nhận trạng thái này và bị hủy học phần, bị xử lý vi phạm theo Quy chế Công tác Sinh viên, bị trừ điểm rèn luyện và không được xét học bổng.


In [21]:
# 7
print(app.rag(source="nttu")("Điều kiện nào để được xét chuyển trường?")["result"])

Processed prompts: 100%|██████████| 1/1 [00:13<00:00, 13.93s/it]

 Để được xét chuyển trường, sinh viên cần đáp ứng các điều kiện sau:

1. Không đang là sinh viên trình độ năm 1 hoặc năm cuối, không thuộc diện bị xem xét buộc thôi học và còn đủ thời gian học tập theo quy định;
2. Sinh viên đạt điều kiện trúng tuyển của chương trình, ngành đào tạo cùng khóa tuyển sinh tại nơi chuyển đến;
3. Nơi chuyển đến có đủ các điều kiện bảo đảm chất lượng, chưa vượt quá năng lực đào tạo đối với chương trình, ngành đào tạo đó theo quy định hiện hành của Bộ GD&ĐT;
4. Được sự đồng ý của thủ trưởng cơ sở đào tạo xin chuyển đến và các cơ sở đào tạo xin chuyển đi;
5. Sinh viên phải hoàn tất các nghĩa vụ với trường đang theo học.

Không được phép chuyển trường trong các trường hợp sau:
1. Sinh viên không đủ điều kiện trúng tuyển vào trường xin chuyển đến;
2. Sinh viên thuộc diện nằm ngoài vùng tuyển quy định của trường xin chuyển đến;
3. Sinh viên năm thứ nhất và năm cuối khóa.

Quy trình: Sinh viên tải mẫu đơn BM-ĐT-12 xin chuyển trường trên website Phòng Quản lý Đào t

In [33]:
# 8
print(app.rag(source="nttu")("Nếu điểm thi kết thúc học phần < 4 thì như thế nào?")["result"])

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.45s/it]

 Nếu điểm thi kết thúc học phần < 4 thì điểm tổng kết học phần lấy bằng điểm thi kết thúc học phần (sinh viên bị rớt môn học).


In [23]:
# 9
print(app.rag(source="nttu")("Phí cấp lại thẻ sinh viên khi bị mất là bao nhiêu?")["result"])

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.28s/it]

 Phí cấp lại thẻ sinh viên khi bị mất là 20.000 đồng/ lần.


In [26]:
# 10
print(app.rag(source="nttu")("Để đạt loại xuất sắc điểm rèn luyện cần bao nhiêu điểm?")["result"])

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.38s/it]

 Để đạt điểm rèn luyện loại xuất sắc, cần đạt điểm rèn luyện từ 90 đến 100 điểm.


In [24]:
# 11
print(app.rag(source="nttu")("Để đạt loại tốt điểm rèn luyện cần bao nhiêu điểm?")["result"])

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.48s/it]

 Để đạt loại tốt điểm rèn luyện, sinh viên cần đạt điểm rèn luyện từ 80 đến dưới 90 điểm.


In [25]:
# 12
print(app.rag(source="nttu")("Nếu sinh viên không đạt ở một học phần, phải làm gì?")["result"])

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.81s/it]

 Nếu sinh viên không đạt ở một học phần, họ phải đăng ký và học lại từ đầu học phần đó.


In [27]:
# 13
print(app.rag(source="nttu")("Lớp trưởng được cấp bao nhiêu tiền?")["result"])

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.64s/it]

 Lớp trưởng được cấp 200.000 đồng/tháng/cá nhân x 10 tháng.


In [28]:
# 14
print(app.rag(source="nttu")("Lớp phó được cấp bao nhiêu tiền?")["result"])

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.43s/it]

 Lớp phó được cấp 100.000 đồng mỗi tháng.


In [29]:
# 15
print(app.rag(source="nttu")("Bao nhiêu điểm thì bị rớt?")["result"])

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.48s/it]

 Điểm thi kết thúc học phần < 4 thì điểm tổng kết học phần lấy bằng điểm thi kết thúc học phần (sinh viên bị rớt môn học).
